# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [14]:
# import libraries
from sqlalchemy import create_engine
import nltk
nltk.download(['punkt', 'wordnet'])

import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
import pickle


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [15]:
engine = create_engine('sqlite:///disaster_messages_database.db')
df = pd.read_sql("SELECT * FROM disaster_messages_table", engine)
X = df.message
y = df[df.columns[4:]]
category_names = y.columns

In [16]:
df.head(1)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
category_names = y.columns

In [18]:
category_names

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'water', 'food', 'shelter', 'clothing', 'money', 'missing_people',
       'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport',
       'buildings', 'electricity', 'tools', 'hospitals', 'shops',
       'aid_centers', 'other_infrastructure', 'weather_related', 'floods',
       'storm', 'fire', 'earthquake', 'cold', 'other_weather',
       'direct_report'],
      dtype='object')

In [19]:
df.describe()

,id,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
count,26216.00000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,...,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000
mean,15224.82133,0.759307,0.170659,0.004501,0.414251,0.079493,0.050084,0.027617,0.017966,0.032804,...,0.011787,0.043904,0.278341,0.082202,0.093187,0.010757,0.093645,0.020217,0.052487,0.193584
std,8826.88914,0.427512,0.376218,0.066940,0.492602,0.270513,0.218122,0.163875,0.132831,0.178128,...,0.107927,0.204887,0.448191,0.274677,0.290700,0.103158,0.291340,0.140743,0.223011,0.395114
min,2.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7446.75000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,15662.50000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,22924.25000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,30265.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


### 2. Write a tokenization function to process your text data

In [20]:
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [21]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
np.random.seed(13)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [23]:
y_test.head(2)

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
23651,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8667,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [24]:
y_pred = pipeline.predict(X_test)

for i in range(36):
    print(y_test.columns[i], ':')
    print(classification_report(y_test.iloc[:,i], y_pred[:,i], target_names=category_names), '...................................................')

related :
                        precision    recall  f1-score   support

               related       0.59      0.37      0.45      1545
               request       0.83      0.92      0.87      5009

           avg / total       0.77      0.79      0.77      6554
 ...................................................
request :
                        precision    recall  f1-score   support

               related       0.88      0.98      0.93      5413
               request       0.82      0.37      0.51      1141

           avg / total       0.87      0.88      0.86      6554
 ...................................................
offer :
                        precision    recall  f1-score   support

               related       1.00      1.00      1.00      6531
               request       0.00      0.00      0.00        23

           avg / total       0.99      1.00      0.99      6554
 ...................................................
aid_related :
                        p

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 2, does not match size of target_names, 35
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


IndexError: index 35 is out of bounds for axis 0 with size 35

In [25]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f1860f77b70>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

### 6. Improve your model
Use grid search to find better parameters. 

In [26]:
parameters = {
    #'vect__ngram_range': ((1, 1),(1,2)),
    #'vect__max_df': (0.5, 0.75, 1.0),
    #'vect__max_features': (None, 5000, 10000),
    #'tfidf__use_idf': (True, False),
    'clf__estimator__n_estimators': [50, 100, 150],
    'clf__estimator__min_samples_split': [2, 3, 4],
    
}

cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs=4, verbose=2)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [27]:
cv.fit(X_train, y_train)

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50 
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50 
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50 
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, total=13.4min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, total=13.5min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, total=13.6min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=150 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, total=26.3min
[CV] clf__estimator__min_samples_split=2, clf__est

[Parallel(n_jobs=4)]: Done  27 out of  27 | elapsed: 176.9min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=4,
       param_grid={'clf__estimator__n_estimators': [50, 100, 150], 'clf__estimator__min_samples_split': [2, 3, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

In [32]:
cv.grid_scores_

/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.23670, std: 0.00693, params: {'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 50},
 mean: 0.23629, std: 0.00782, params: {'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 100},
 mean: 0.23812, std: 0.00673, params: {'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 150},
 mean: 0.22958, std: 0.00424, params: {'clf__estimator__min_samples_split': 3, 'clf__estimator__n_estimators': 50},
 mean: 0.23772, std: 0.00928, params: {'clf__estimator__min_samples_split': 3, 'clf__estimator__n_estimators': 100},
 mean: 0.23807, std: 0.00820, params: {'clf__estimator__min_samples_split': 3, 'clf__estimator__n_estimators': 150},
 mean: 0.23090, std: 0.00712, params: {'clf__estimator__min_samples_split': 4, 'clf__estimator__n_estimators': 50},
 mean: 0.23360, std: 0.00865, params: {'clf__estimator__min_samples_split': 4, 'clf__estimator__n_estimators': 100},
 mean: 0.23446, std: 0.00820, params: {'clf__estimator__min_samples

In [33]:
print(cv.best_params_)

{'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 150}


In [34]:
#building new model
optimised_model = cv.best_estimator_
print (cv.best_estimator_)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [35]:
y_pred = optimised_model.predict(X_test)

for i in range(36):
    print(y_test.columns[i], ':')
    print(classification_report(y_test.iloc[:,i], y_pred[:,i]), '...................................................')

related :
             precision    recall  f1-score   support

          0       0.75      0.31      0.44      1545
          1       0.82      0.97      0.89      5009

avg / total       0.80      0.81      0.78      6554
 ...................................................
request :
             precision    recall  f1-score   support

          0       0.89      0.99      0.94      5413
          1       0.89      0.41      0.56      1141

avg / total       0.89      0.89      0.87      6554
 ...................................................
offer :
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6531
          1       0.00      0.00      0.00        23

avg / total       0.99      1.00      0.99      6554
 ...................................................
aid_related :
             precision    recall  f1-score   support

          0       0.76      0.88      0.81      3821
          1       0.78      0.61      0.68      27

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


IndexError: index 35 is out of bounds for axis 0 with size 35

### 9. Export your model as a pickle file

In [36]:
with open('MLclassifier.pkl', 'wb') as file:
    pickle.dump(optimised_model, file)

In [37]:
m = pickle.dumps('classifier.pkl')

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.